In [1]:
#import sqlalchemy
import sqlalchemy as sqa
from sqlalchemy.orm import declarative_base

from sqlalchemy.orm import sessionmaker

In [2]:
#creates engine to the database file
engine = sqa.create_engine("sqlite:///Data/movie.db")

In [3]:
#creates declarative base
Base = declarative_base()

In [4]:
#creates class for communicating with the movie table
class Movies(Base):
    __tablename__ = 'movies'
    
    id_movie = sqa.Column(sqa.Integer, primary_key=True)
    title_movie = sqa.Column(sqa.String)
    
    
    def __repr__(self):
        return "<Movie(id_movie='%i',title_movie='%s')>" % (self.id_movie, self.title_movie)

#creates class for communicating with the movie table
class Ratings(Base):
    __tablename__ = 'ratings'
    
    id_rating = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    id_user = sqa.Column(sqa.Integer)
    value_rating = sqa.Column(sqa.Float)
    timestamp_rating = sqa.Column(sqa.String)

    
    def __repr__(self):
        return "<Rating(id_rating='%i',id_movie='%i',id_user='%i',value_rating='%i',timestamp_rating='%s')>" % (self.id_rating,self.id_movie,self.id_user,self.value_rating, self.timestamp_rating)
    
#creates class for communicating with the movie table
class GenreList(Base):
    __tablename__ = 'genre_list'
    
    id_genre_item = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    name_genre = sqa.Column(sqa.String)
    
    
    def __repr__(self):
        return "<GenreList(id_genre_item='%i',id_movie='%i',name_genre='%s')>" % (self.id_genre_item,self.id_movie,self.id_user,self.name_genre)

In [5]:
#connects to the database
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
for m in session.query(Movies).order_by(Movies.id_movie)[1:30]:
    print(m.title_movie)

Jumanji (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
Sabrina (1995)
Tom and Huck (1995)
Sudden Death (1995)
GoldenEye (1995)
American President, The (1995)
Dracula: Dead and Loving It (1995)
Balto (1995)
Nixon (1995)
Cutthroat Island (1995)
Casino (1995)
Sense and Sensibility (1995)
Four Rooms (1995)
Ace Ventura: When Nature Calls (1995)
Money Train (1995)
Get Shorty (1995)
Copycat (1995)
Assassins (1995)
Powder (1995)
Leaving Las Vegas (1995)
Othello (1995)
Now and Then (1995)
Persuasion (1995)
City of Lost Children, The (Cité des enfants perdus, La) (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)


In [30]:
for m in (session.query(Movies).filter(Movies.title_movie.like("%Jumanji%Welcome%"))[0:30]):
    print(m.title_movie)

Jumanji: Welcome to the Jungle (2017)


In [ ]:
.filter(m.title_movie.like("%Jumanji%").order_by(Movies.id_movie)